In [13]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GATConv
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE


In [14]:
dataset = Planetoid(root='../data', name='Cora')
data = dataset[0].to(device)

Processing...
/Users/adityaprathapa/graph-attention-networks/venv/lib/python3.12/site-packages/torch_geometric/io/planetoid.py:107: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  out = pickle.load(f, encoding='latin1')
/Users/adityaprathapa/graph-attention-networks/venv/lib/python3.12/site-packages/torch_geometric/io/planetoid.py:107: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  out = pickle.load(f, encoding='latin1')
/Users/adityaprathapa/graph-attention-networks/venv/lib/python3.12/site-packages/torch_geometric/io/planetoid.py:107: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2

In [16]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=8, dropout=0.6):
        super().__init__()
        self.dropout = dropout

        # Layer 1: 8 attention heads, each producing 8 features
        # Output: 64 features (concatenated)
        self.conv1 = GATConv(
            in_channels, hidden_channels, heads=heads, dropout=dropout
        )

        # Layer 2: 1 attention head for classification
        # Input: 64 (from concat above)
        # Output: 7 classes
        self.conv2 = GATConv(
            hidden_channels * heads, out_channels, heads=1, concat=False, dropout=dropout
        )

    def forward(self, x, edge_index):
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

    def get_embeddings(self, x, edge_index):
        """Extract layer-1 embeddings for t-SNE visualization later."""
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        return x


model = GAT(
    in_channels=dataset.num_features,   
    hidden_channels=8,                   
    out_channels=dataset.num_classes,    
    heads=8,
    dropout=0.6
).to(device)